In [21]:
"""
    Pappus' creativity is due to him (implicitly) distinguishing between numerical identity and conceptual identity
    in step 1 and applying this distinction to the triangle ABC to take advantage of Euclid I.4.
"""

!python --version

# install and import packages
!pip install rdflib

import glob
import os
import rdflib
import requests
import typing

!pwd

Python 3.12.2
/Users/matteobianchetti/Documents/Chen.Chen.1/Filosofia/Logica/Mathematics/Math_subjects/Computer_science/Git_repositories/GitHub/ontology-and-proofs/tools/venv/code


# PROOFS OF EUCLID I.5

In [31]:
# helper function to raise exception if the REST endpoint returns an
# unexpected status code
# def check_response_ok(response, message):
#     if not response.ok:
#         raise Exception(
#             f"{message}\nStatus received={response.status_code}\n{response.text}")

sparql_text_0:str = """
        select distinct 
        ?p
        ?label
        where {
        ?s ?p ?o .
        ?p rdfs:label ?label
        }
        order by ?label
        limit 2
    """

# submit sparql query and retrieve results
def query_ontology(sparql_query:str,
                  rdfox_server:str = "http://localhost:12110",
                  datastore:str = "/datastores/foom_pappus/sparql") -> str:
    # query the datastore at the rdfox server
    response = requests.get(f"{rdfox_server}{datastore}", params={"query": sparql_query})
    # check if the REST endpoint returns an unexpected status code
    if not response.ok:
        return ""
    # if the REST endpoint returns an expected response, return the text of the response
    return response.text

print(query_ontology(sparql_text_0))
results = query_ontology(sparql_text_0)
print(results)
results

def collect_results(response:str) -> dict:
    results:dict = {}
    if response:
        pass
    return results


# def collect_proofs(sparql_query:str = sparql_query_find_proofs()) -> dict:
#     proofs:dict = {}
#     response = query_ontology(sparql_query)
#     if response:
#         for response.
#     return proofs

?p	?label
<http://www.foom.com/core#00000000000000000170>	"construction satisfies request"@en
<http://www.foom.com/mathematical_concepts#00000000000000000163>	"defines"@en

?p	?label
<http://www.foom.com/core#00000000000000000170>	"construction satisfies request"@en
<http://www.foom.com/mathematical_concepts#00000000000000000163>	"defines"@en



'?p\t?label\n<http://www.foom.com/core#00000000000000000170>\t"construction satisfies request"@en\n<http://www.foom.com/mathematical_concepts#00000000000000000163>\t"defines"@en\n'

In [ ]:
#### SPARQL QUERIES
# find all proofs
def sparql_query_find_proofs() -> str:
    sparql_query:str = """
        SELECT DISTINCT 
            ?proof_iri
            ?proof_label
        WHERE {
            ?proof_iri 
                a <http://www.foom.com/core/Proof> ;
                rdfs:label ?proof_label .
        }
    """
    return sparql_query



In [ ]:
# 0. select a proof
# find all proofs


# 1. statement to prove


# 2. conceptual items connected to that statement
# 3. apply heuristics
# 4. compare step with results of heuristics
# 5. repeat for each step



In [19]:
#### SPARQL QUERIES

# find all proofs
def sparql_query_find_proofs():
    sparql_query:str = """
        SELECT DISTINCT 
            ?proof_iri
            ?proof_label
        WHERE {
            ?proof_iri 
                a <http://www.foom.com/core/Proof> ;
                rdfs:label ?proof_label .
        }
    """
    return sparql_query

# for every proof, find all the steps in that proof
def sparql_query_find_steps_in_proof(given_proof_iri:str):
    sparql_query:str = f"""
    SELECT DISTINCT
        ?proof_iri
        ?proof_step_iri
        ?proof_step_label
        ?antecendent_proof_step_iri
    WHERE {{
        ?proof_iri 
            a <http://www.foom.com/core/Proof> .
        ?proof_step_iri
            a <http://www.foom.com/core/Proof_step> ;
            rdfs:label ?proof_step_label ;
            <http://www.foom.com/core/inProof> {given_proof_iri} .
        OPTIONAL {{
            ?proof_step_iri <http://www.foom.com/core/hasAntecedent> ?antecendent_proof_step_iri .
        }}
    }}
    """
    return sparql_query

# find all the individuals and properties used in the proof steps of a given proof
# group by proof step
def sparql_query_find_individuals_and_properties_of_proof_steps_of_given_proof(given_proof_iri:str):
    sparql_query:str = f"""
    SELECT DISTINCT
        ?proof_iri
        ?proof_step_iri
    WHERE {{
        ?proof_iri 
            a <http://www.foom.com/core/Proof> .
        ?proof_step_iri
            a <http://www.foom.com/core/Proof_step> ;
            <http://www.foom.com/core/inProof> {given_proof_iri} .
        
    }}
    """
    return sparql_query

# find exactly all the properties used in some proof step
def sparql_query_find_properties_proof_steps():
    sparql_query:str = """
        SELECT DISTINCT 
            ?p 
            ?p_label
            ?p_type
        WHERE {
            ?proof_step_iri 
                a <http://www.foom.com/core/Proof_step> ;
                ?p ?o .
            ?p 
                rdfs:label ?p_label ;
                a ?p_type .
        }
    """
    return sparql_query
    

# for every step, find the triple attached to it and the items to which the step directly refers

